In [1]:
import requests
import json
import base64
import PIL
from PIL import Image
import pandas as pd
import time
import statsmodels.api as sm

/Users/leevzkaplan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [55]:
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stylegan_image:
         #read the image
        im = Image.open(f"stock_images/{stock_image}")
        #get the ratio, make sure longest edge is 2000 pixels and shorter edge is proportional
        width, height = im.size
        print(f"Old dimensions: ({width}, {height})")
        if width > height:
            ratio = height / width
            new_width = 2000
            new_height = int(ratio * new_width)
        else:
            ratio = width / height
            new_height = 2000
            new_width = int(ratio * new_height)
        newsize = (new_width, new_height)
        im = im.resize(newsize)
        print(f"New dimensions: ({new_width}, {new_height})")
        im.save(f"stock_images/{stock_image}")

Old dimensions: (3231, 3231)
New dimensions: (2000, 2000)
Old dimensions: (3196, 3196)
New dimensions: (2000, 2000)
Old dimensions: (1984, 1984)
New dimensions: (2000, 2000)
Old dimensions: (4106, 4106)
New dimensions: (2000, 2000)
Old dimensions: (2824, 2818)
New dimensions: (2000, 1995)
Old dimensions: (2643, 2643)
New dimensions: (2000, 2000)
Old dimensions: (2875, 2859)
New dimensions: (2000, 1988)
Old dimensions: (4158, 4237)
New dimensions: (1962, 2000)
Old dimensions: (3374, 3374)
New dimensions: (2000, 2000)
Old dimensions: (2533, 2533)
New dimensions: (2000, 2000)
Old dimensions: (2820, 2820)
New dimensions: (2000, 2000)
Old dimensions: (3313, 3313)
New dimensions: (2000, 2000)
Old dimensions: (6438, 5187)
New dimensions: (2000, 1611)
Old dimensions: (2635, 2635)
New dimensions: (2000, 2000)
Old dimensions: (3035, 3035)
New dimensions: (2000, 2000)
Old dimensions: (4075, 3967)
New dimensions: (2000, 1946)
Old dimensions: (1943, 1943)
New dimensions: (2000, 2000)
Old dimensions

In [84]:

BASE_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'

def detect(image_path):
    """
    This function uses HTTP POST to query the Face++ API 
    that detects and classifies faces in the given image
    
    Args:
        image_path: path to the image that we want to classify
        
    Returns:
        result: a dictionary containing the API response
    """

    #read the image
    im = Image.open(image_path)
    #flip image
    out = im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    split = image_path.split("/")
    split[-1] = "transposed-" + split[-1]
    transp_image_path = "/".join(split)
    out.save(transp_image_path)
    
    
    def get_call(image_path): 
        with open(image_path, 'rb') as image_file:
            # encode in base64
            image_base64 = base64.b64encode(image_file.read())
            
            return {'api_key': "AJUC6jZcYaEtVm62lA944PD18g8qzLn-",
                'api_secret': "NiML8ojHjroKMog2AKvzMuu5PpaJhDP2",
                'image_base64': image_base64,
                'return_attributes': 'gender,age'} 
    
    
    original_response = requests.post(url = BASE_URL, data = get_call(image_path))
    flipped_response = requests.post(url = BASE_URL, data = get_call(transp_image_path))
    return json.loads(original_response.content), json.loads(flipped_response.content)

In [85]:
def get_dems_of_row(row_name):
    if row_name[-9] == "2":
        race = "black"
    else:
        race = "white"
    if row_name[-7] == "2":
        gender = " male"
    else:
        gender = " female"
    if row_name[-5] == "0":
        age = "child"
    elif row_name[-5] == "1":
        age = "teen"
    elif row_name[-5] == "2":
        age = "adult"
    elif row_name[-5] == "3":
        age = "middle-age"
    else:
        age = "old"
    return gender + " " + race + " " + age

In [86]:
stylegan_images = []
stock_images = []
for stylegan_image in os.listdir("stylegan_images"):
    if stylegan_image != ".DS_Store":
        stylegan_images.append(stylegan_image)
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store":
        stock_images.append(stock_image)

In [87]:
image_dem = []
for ad in pd.Series(stylegan_images):
    new_name = get_dems_of_row(ad)
    image_dem.append("stylegan" + new_name)
for ad in pd.Series(stock_images):
    new_name = get_dems_of_row(ad)
    image_dem.append("stock" + new_name)

In [88]:
sg_original_ages = []
sg_original_genders = []
sg_y_flipped_ages = []
sg_y_flipped_genders = []
for stylegan_image in os.listdir("stylegan_images"):
    if stylegan_image != ".DS_Store" and "transposed" not in stylegan_image:
        original, flipped  = detect(f"stylegan_images/{stylegan_image}")
        sg_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        sg_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        sg_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        sg_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        print(len(sg_original_ages))
        time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [89]:
stock_original_ages = []
stock_original_genders = []
stock_y_flipped_ages = []
stock_y_flipped_genders = []
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stock_image:
        original, flipped  = detect(f"stock_images/{stock_image}")
        stock_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        stock_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        stock_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        stock_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        print(len(stock_original_ages))
        time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [90]:
images = stylegan_images
images = stylegan_images + stock_images

original_ages = sg_original_ages + stock_original_ages

original_genders = sg_original_genders + stock_original_genders

y_flipped_ages = sg_y_flipped_ages + stock_y_flipped_ages

y_flipped_genders  = sg_y_flipped_genders + stock_y_flipped_genders
len(original_ages)

200

In [93]:
images = stylegan_images + stock_images

In [103]:
stock_images_copy = []
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stock_image:
        stock_images_copy.append(stock_image)
len(stock_images_copy)

100

In [104]:
images = stylegan_images + stock_images_copy
len(images)

200

In [106]:
len(original_ages)

200

In [107]:
df = pd.DataFrame()
df["Images"] = images
df["Original Age"] = original_ages
df["Original Gender"] = original_genders
df["Y-Axis Flip Age"] = y_flipped_ages
df["Y-Axis Flip Gender"] = y_flipped_genders
df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,1628188590507_2_2_3.jpg,66,Male,72,Male
1,1628188590507_2_0_1.jpg,21,Female,22,Female
2,1628188590507_2_0_0.jpg,20,Female,20,Female
3,1628188590507_2_2_2.jpg,31,Male,31,Male
4,1628188590507_2_2_0.jpg,19,Female,17,Male
...,...,...,...,...,...
195,a_0_2_1.jpg,21,Male,21,Male
196,a_0_2_3.jpg,53,Male,51,Male
197,a_0_0_1.jpg,25,Female,29,Female
198,a_0_0_0.jpg,34,Female,36,Female


In [108]:
df.to_csv("y_axis_flip_data.csv")

In [4]:
df = pd.read_csv("y_axis_flip_data.csv", index_col=0)
df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,1628188590507_2_2_3.jpg,66,Male,72,Male
1,1628188590507_2_0_1.jpg,21,Female,22,Female
2,1628188590507_2_0_0.jpg,20,Female,20,Female
3,1628188590507_2_2_2.jpg,31,Male,31,Male
4,1628188590507_2_2_0.jpg,19,Female,17,Male
...,...,...,...,...,...
195,a_0_2_1.jpg,21,Male,21,Male
196,a_0_2_3.jpg,53,Male,51,Male
197,a_0_0_1.jpg,25,Female,29,Female
198,a_0_0_0.jpg,34,Female,36,Female


In [5]:
df["Age Difference"] = df["Original Age"] / df["Y-Axis Flip Age"]
df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference
0,1628188590507_2_2_3.jpg,66,Male,72,Male,0.916667
1,1628188590507_2_0_1.jpg,21,Female,22,Female,0.954545
2,1628188590507_2_0_0.jpg,20,Female,20,Female,1.000000
3,1628188590507_2_2_2.jpg,31,Male,31,Male,1.000000
4,1628188590507_2_2_0.jpg,19,Female,17,Male,1.117647
...,...,...,...,...,...,...
195,a_0_2_1.jpg,21,Male,21,Male,1.000000
196,a_0_2_3.jpg,53,Male,51,Male,1.039216
197,a_0_0_1.jpg,25,Female,29,Female,0.862069
198,a_0_0_0.jpg,34,Female,36,Female,0.944444


In [6]:
original_gender_to_int = [1 if x == "Male" else -1 for x in df["Original Gender"]]
flip_gender_to_int = [-1 if x == "Male" else 1 for x in df["Y-Axis Flip Gender"]]
df["Gender Difference"] = [a + b for a,b in zip(original_gender_to_int, flip_gender_to_int)]
df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,1628188590507_2_2_3.jpg,66,Male,72,Male,0.916667,0
1,1628188590507_2_0_1.jpg,21,Female,22,Female,0.954545,0
2,1628188590507_2_0_0.jpg,20,Female,20,Female,1.000000,0
3,1628188590507_2_2_2.jpg,31,Male,31,Male,1.000000,0
4,1628188590507_2_2_0.jpg,19,Female,17,Male,1.117647,-2
...,...,...,...,...,...,...,...
195,a_0_2_1.jpg,21,Male,21,Male,1.000000,0
196,a_0_2_3.jpg,53,Male,51,Male,1.039216,0
197,a_0_0_1.jpg,25,Female,29,Female,0.862069,0
198,a_0_0_0.jpg,34,Female,36,Female,0.944444,0


In [29]:
df.to_csv("y_axis_flip_data_with_targets.csv")

In [37]:
df = pd.read_csv("y_axis_flip_data_with_targets.csv", index_col=0)

In [38]:
style_gan_df = df[:100]
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,1628188590507_2_2_3.jpg,66,Male,72,Male,0.916667,0
1,1628188590507_2_0_1.jpg,21,Female,22,Female,0.954545,0
2,1628188590507_2_0_0.jpg,20,Female,20,Female,1.000000,0
3,1628188590507_2_2_2.jpg,31,Male,31,Male,1.000000,0
4,1628188590507_2_2_0.jpg,19,Female,17,Male,1.117647,-2
...,...,...,...,...,...,...,...
95,1628188536814_0_2_4.jpg,80,Male,77,Male,1.038961,0
96,1628188536814_0_0_4.jpg,83,Female,83,Female,1.000000,0
97,1628188755886_2_0_4.jpg,75,Female,73,Female,1.027397,0
98,1628188704081_2_0_4.jpg,82,Male,81,Male,1.012346,0


In [39]:
stylegan_changed_age = (style_gan_df["Original Age"] != style_gan_df["Y-Axis Flip Age"]).mean()
stylegan_changed_age

0.81

In [40]:
stylegan_changed_gender = (style_gan_df["Original Gender"] != style_gan_df["Y-Axis Flip Gender"]).mean()
stylegan_changed_gender

0.06

In [41]:
stock_df = df[100:]
stock_df = stock_df.reset_index(drop=True)
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,c_2_0_3.jpg,36,Female,44,Female,0.818182,0
1,c_2_2_1.jpg,22,Female,23,Female,0.956522,0
2,d_0_0_2.jpg,48,Female,47,Female,1.021277,0
3,e_2_0_4.jpg,80,Female,64,Female,1.250000,0
4,d_0_2_0.jpg,16,Male,17,Male,0.941176,0
...,...,...,...,...,...,...,...
95,a_0_2_1.jpg,21,Male,21,Male,1.000000,0
96,a_0_2_3.jpg,53,Male,51,Male,1.039216,0
97,a_0_0_1.jpg,25,Female,29,Female,0.862069,0
98,a_0_0_0.jpg,34,Female,36,Female,0.944444,0


In [42]:
stock_changed_age = (stock_df["Original Age"] != stock_df["Y-Axis Flip Age"]).mean()
stock_changed_age

0.84

In [43]:
stock_changed_gender = (stock_df["Original Gender"] != stock_df["Y-Axis Flip Gender"]).mean()
stock_changed_gender

0.06

In [44]:
def get_dems_of_row(row_name):
    if row_name[-9] == "2":
        race = "black"
    else:
        race = "white"
    if row_name[-7] == "2":
        gender = " male"
    else:
        gender = " female"
    if row_name[-5] == "0":
        age = "child"
    elif row_name[-5] == "1":
        age = "teen"
    elif row_name[-5] == "2":
        age = "adult"
    elif row_name[-5] == "3":
        age = "middle-age"
    else:
        age = "old"
    return gender + " " + race + " " + age


def rename_index(df):
    index = []
    for ad in pd.Series(df["Images"]):
        new_name = get_dems_of_row(ad)
        index.append(new_name)

    df["Images"] = index

rename_index(style_gan_df)
rename_index(stock_df)

<ipython-input-44-36973e748560>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Images"] = index


In [45]:
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,male black middle-age,66,Male,72,Male,0.916667,0
1,female black teen,21,Female,22,Female,0.954545,0
2,female black child,20,Female,20,Female,1.000000,0
3,male black adult,31,Male,31,Male,1.000000,0
4,male black child,19,Female,17,Male,1.117647,-2
...,...,...,...,...,...,...,...
95,male white old,80,Male,77,Male,1.038961,0
96,female white old,83,Female,83,Female,1.000000,0
97,female black old,75,Female,73,Female,1.027397,0
98,female black old,82,Male,81,Male,1.012346,0


In [46]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,female black middle-age,36,Female,44,Female,0.818182,0
1,male black teen,22,Female,23,Female,0.956522,0
2,female white adult,48,Female,47,Female,1.021277,0
3,female black old,80,Female,64,Female,1.250000,0
4,male white child,16,Male,17,Male,0.941176,0
...,...,...,...,...,...,...,...
95,male white teen,21,Male,21,Male,1.000000,0
96,male white middle-age,53,Male,51,Male,1.039216,0
97,female white teen,25,Female,29,Female,0.862069,0
98,female white child,34,Female,36,Female,0.944444,0


In [47]:
def determine_true_age(df):
    def age_bucket(age_str):
        if "child" in age_str:
            return "0-12"
        elif "teen" in age_str:
            return "12-18"
        elif "adult" in age_str:
            return "19-34"
        elif "middle-age" in age_str:
            return "35-54"
        else:
            return "65+"
    df["True Age"] = df["Images"].map(age_bucket)
    return df

stock_df = determine_true_age(stock_df)
style_gan_df = determine_true_age(style_gan_df)

        

<ipython-input-47-c5db7548e2d6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["True Age"] = df["Images"].map(age_bucket)


In [48]:
def determine_true_gender(df):
    def gender(g_str):
        if ("female" in g_str):
            return "Female"
        else:
            return "Male"
    df["True Gender"] = df["Images"].map(gender)
    return df

stock_df = determine_true_gender(stock_df)
style_gan_df = determine_true_gender(style_gan_df)

<ipython-input-48-c25a830c718f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["True Gender"] = df["Images"].map(gender)


In [49]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference,True Age,True Gender
0,female black middle-age,36,Female,44,Female,0.818182,0,35-54,Female
1,male black teen,22,Female,23,Female,0.956522,0,12-18,Male
2,female white adult,48,Female,47,Female,1.021277,0,19-34,Female
3,female black old,80,Female,64,Female,1.250000,0,65+,Female
4,male white child,16,Male,17,Male,0.941176,0,0-12,Male
...,...,...,...,...,...,...,...,...,...
95,male white teen,21,Male,21,Male,1.000000,0,12-18,Male
96,male white middle-age,53,Male,51,Male,1.039216,0,35-54,Male
97,female white teen,25,Female,29,Female,0.862069,0,12-18,Female
98,female white child,34,Female,36,Female,0.944444,0,0-12,Female


In [50]:
def generate_age_buckets(df):
    def age_bucket(num):
        if num < 12:
            return "0-12"
        elif num < 18:
            return "12-18"
        elif num < 34:
            return "19-34"
        elif num < 54:
            return "35-54"
        else:
            return "65+"
    df["Original Age Bucket"] = df["Original Age"].map(age_bucket)
    df["Y-Axis Flip Age Bucket"] = df["Y-Axis Flip Age"].map(age_bucket)
    return df

stock_df = generate_age_buckets(stock_df)
style_gan_df = generate_age_buckets(style_gan_df)

<ipython-input-50-bf728aea5615>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Original Age Bucket"] = df["Original Age"].map(age_bucket)
<ipython-input-50-bf728aea5615>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Y-Axis Flip Age Bucket"] = df["Y-Axis Flip Age"].map(age_bucket)


In [51]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference,True Age,True Gender,Original Age Bucket,Y-Axis Flip Age Bucket
0,female black middle-age,36,Female,44,Female,0.818182,0,35-54,Female,35-54,35-54
1,male black teen,22,Female,23,Female,0.956522,0,12-18,Male,19-34,19-34
2,female white adult,48,Female,47,Female,1.021277,0,19-34,Female,35-54,35-54
3,female black old,80,Female,64,Female,1.250000,0,65+,Female,65+,65+
4,male white child,16,Male,17,Male,0.941176,0,0-12,Male,12-18,12-18
...,...,...,...,...,...,...,...,...,...,...,...
95,male white teen,21,Male,21,Male,1.000000,0,12-18,Male,19-34,19-34
96,male white middle-age,53,Male,51,Male,1.039216,0,35-54,Male,35-54,35-54
97,female white teen,25,Female,29,Female,0.862069,0,12-18,Female,19-34,19-34
98,female white child,34,Female,36,Female,0.944444,0,0-12,Female,35-54,35-54


In [52]:
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference,True Age,True Gender,Original Age Bucket,Y-Axis Flip Age Bucket
0,male black middle-age,66,Male,72,Male,0.916667,0,35-54,Male,65+,65+
1,female black teen,21,Female,22,Female,0.954545,0,12-18,Female,19-34,19-34
2,female black child,20,Female,20,Female,1.000000,0,0-12,Female,19-34,19-34
3,male black adult,31,Male,31,Male,1.000000,0,19-34,Male,19-34,19-34
4,male black child,19,Female,17,Male,1.117647,-2,0-12,Male,19-34,12-18
...,...,...,...,...,...,...,...,...,...,...,...
95,male white old,80,Male,77,Male,1.038961,0,65+,Male,65+,65+
96,female white old,83,Female,83,Female,1.000000,0,65+,Female,65+,65+
97,female black old,75,Female,73,Female,1.027397,0,65+,Female,65+,65+
98,female black old,82,Male,81,Male,1.012346,0,65+,Female,65+,65+


In [53]:
def get_acc(df):
    return (df["Y-Axis Flip Age Bucket"] == df["True Age"]).mean() - (df["Original Age Bucket"] == df["True Age"]).mean()

In [58]:
def get_gender_acc(df):
    return (df["Y-Axis Flip Gender"] == df["True Gender"]).mean() - (df["Original Gender"] == df["True Gender"]).mean()

In [22]:
get_acc(stock_df)

0.07

In [23]:
get_acc(style_gan_df)

0.010000000000000009

In [24]:
style_gan_df.describe()

,Original Age,Y-Axis Flip Age,Age Difference,Gender Difference
count,100.000000,100.000000,100.000000,100.000000
mean,45.060000,45.140000,0.993054,0.040000
std,26.041079,25.842186,0.090762,0.490722
min,6.000000,7.000000,0.615385,-2.000000
25%,20.750000,21.000000,0.950000,0.000000
50%,36.500000,36.500000,1.000000,0.000000
75%,73.250000,72.250000,1.047619,0.000000
max,87.000000,89.000000,1.181818,2.000000


In [25]:
stock_df.describe()

,Original Age,Y-Axis Flip Age,Age Difference,Gender Difference
count,100.000000,100.000000,100.000000,100.000000
mean,40.010000,39.810000,1.026389,0.000000
std,21.500526,21.464886,0.164699,0.492366
min,6.000000,4.000000,0.550000,-2.000000
25%,22.000000,22.750000,0.954281,0.000000
50%,32.500000,31.500000,1.000000,0.000000
75%,54.750000,55.250000,1.073954,0.000000
max,91.000000,90.000000,2.000000,2.000000


In [70]:
def analysis(df, year_threshold = 5):
    changed_age = (df["Original Age"] != df["Y-Axis Flip Age"]).mean()
    year_change = (abs(df["Original Age"] - df["Y-Axis Flip Age"]) > year_threshold).mean()
    changed_gender = (df["Original Gender"] != df["Y-Axis Flip Gender"]).mean()
    acc = get_acc(df)
    gender_acc = get_gender_acc(df)
    print("Changed Age: ", changed_age)
    print("Year over threshold: ", year_change)
    print("Changed Gender: ", changed_gender)
    print("Overall accuracy: ", acc)
    print("Overall gender accuracy: ", gender_acc)

In [71]:
analysis(stock_df)

Changed Age:  0.84
Year over threshold:  0.18
Changed Gender:  0.06
Overall accuracy:  0.07
Overall gender accuracy:  -0.020000000000000018


In [72]:
analysis(style_gan_df)

Changed Age:  0.81
Year over threshold:  0.08
Changed Gender:  0.06
Overall accuracy:  0.010000000000000009
Overall gender accuracy:  0.0


In [73]:
black_stock_df = stock_df.loc[stock_df["Images"].str.contains("black")].reset_index()
white_stock_df = stock_df.loc[stock_df["Images"].str.contains("white")].reset_index()
black_stylegan_df = stock_df.loc[style_gan_df["Images"].str.contains("black")].reset_index()
white_stylegan_df = stock_df.loc[style_gan_df["Images"].str.contains("white")].reset_index()

In [74]:
analysis(black_stock_df)

Changed Age:  0.86
Year over threshold:  0.2
Changed Gender:  0.06
Overall accuracy:  0.08000000000000002
Overall gender accuracy:  -0.020000000000000018


In [75]:
analysis(white_stock_df)

Changed Age:  0.82
Year over threshold:  0.16
Changed Gender:  0.06
Overall accuracy:  0.06000000000000005
Overall gender accuracy:  -0.020000000000000018


In [76]:
analysis(black_stylegan_df)

Changed Age:  0.86
Year over threshold:  0.22
Changed Gender:  0.06
Overall accuracy:  0.12
Overall gender accuracy:  0.020000000000000018


In [77]:
analysis(white_stylegan_df)

Changed Age:  0.82
Year over threshold:  0.14
Changed Gender:  0.06
Overall accuracy:  0.020000000000000018
Overall gender accuracy:  -0.06000000000000005


In [34]:
lin_reg_features = stock_df

In [58]:
linear_regression_df = pd.DataFrame()
linear_regression_df["ad_image"] = lin_reg_features["Images"]
linear_regression_df["Race"] = ""
linear_regression_df["Age"] = ""
linear_regression_df

,ad_image,Race,Age
0,female black middle-age,,
1,male black teen,,
2,female white adult,,
3,female black old,,
4,male white child,,
...,...,...,...
95,male white teen,,
96,male white middle-age,,
97,female white teen,,
98,female white child,,


In [59]:
linear_regression_df["Gender"] = linear_regression_df.ad_image.apply(lambda x: x.split()[0])
linear_regression_df["Race"] = linear_regression_df.ad_image.apply(lambda x: x.split()[1])
linear_regression_df["Age"] = linear_regression_df.ad_image.apply(lambda x: x.split()[2])
linear_regression_df

,ad_image,Race,Age,Gender
0,female black middle-age,black,middle-age,female
1,male black teen,black,teen,male
2,female white adult,white,adult,female
3,female black old,black,old,female
4,male white child,white,child,male
...,...,...,...,...
95,male white teen,white,teen,male
96,male white middle-age,white,middle-age,male
97,female white teen,white,teen,female
98,female white child,white,child,female


In [60]:
race_one_hot = pd.get_dummies(linear_regression_df.Race)
linear_regression_df = linear_regression_df.join(race_one_hot)

gender_one_hot = pd.get_dummies(linear_regression_df.Gender)
linear_regression_df = linear_regression_df.join(gender_one_hot)

age_one_hot = pd.get_dummies(linear_regression_df.Age)
linear_regression_df = linear_regression_df.join(age_one_hot)
linear_regression_df = linear_regression_df.iloc[: , 4:]
del linear_regression_df["white"]
del linear_regression_df["male"]
del linear_regression_df["adult"]
linear_regression_df

,black,female,child,middle-age,old,teen
0,1,1,0,1,0,0
1,1,0,0,0,0,1
2,0,1,0,0,0,0
3,1,1,0,0,1,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
95,0,0,0,0,0,1
96,0,0,0,1,0,0
97,0,1,0,0,0,1
98,0,1,1,0,0,0


In [61]:
linear_regression_df["Age Difference"] = df["Age Difference"]
linear_regression_df["Gender Difference"] = df["Gender Difference"]
linear_regression_df

,black,female,child,middle-age,old,teen,Age Difference,Gender Difference
0,1,1,0,1,0,0,0.916667,0
1,1,0,0,0,0,1,0.954545,0
2,0,1,0,0,0,0,1.000000,0
3,1,1,0,0,1,0,1.000000,0
4,0,0,1,0,0,0,1.117647,-2
...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,1,1.038961,0
96,0,0,0,1,0,0,1.000000,0
97,0,1,0,0,0,1,1.027397,0
98,0,1,1,0,0,0,1.012346,0


In [74]:
linear_regression_df[linear_regression_df["black"] == 0].describe()

,black,female,child,middle-age,old,teen,Age Difference,Gender Difference
count,50.0,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.0,0.500000,0.200000,0.200000,0.200000,0.200000,0.998054,-0.040000
std,0.0,0.505076,0.404061,0.404061,0.404061,0.404061,0.077515,0.493219
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,-2.000000
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.951510,0.000000
50%,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.039740,0.000000
max,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.166667,2.000000


In [62]:
features = linear_regression_df.iloc[:, :-2]
target = linear_regression_df["Age Difference"]
features

,black,female,child,middle-age,old,teen
0,1,1,0,1,0,0
1,1,0,0,0,0,1
2,0,1,0,0,0,0
3,1,1,0,0,1,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
95,0,0,0,0,0,1
96,0,0,0,1,0,0
97,0,1,0,0,0,1
98,0,1,1,0,0,0


In [67]:
constant_features = sm.add_constant(features, prepend=True)
model = sm.OLS(target, constant_features).fit()
predictions = model.predict(constant_features) 

print_model = model.summary()
print(predictions)

0     0.36
1     0.10
2    -0.10
3     0.26
4    -0.26
      ... 
95   -0.06
96    0.04
97    0.10
98   -0.10
99   -0.26
Length: 100, dtype: float64


In [66]:
target = linear_regression_df["Gender Difference"]
log_reg = sm.Logit(target, features).fit()
print(log_reg.summary())

ValueError: endog must be in the unit interval.